# Hyper Parameter Tune

In this notebook we'll close the cycle the best parameters for the model that we built in previously. This process is called **Hyper Parameter Optimization** and involves some steps:

1. Choose a optimization metric.
2. Create a objective function to minimize/maximize.
3. Define a parameter search space.
4. Execute the paramter search.
5. Check the results. If the found parameters gives a satisfactory model performance, then we are done and proceed to use the model. Otherwise, get back to step 3 and repeat util you find a model that best suits your needs.

In this project will the [Optuna Framework](https://optuna.org/), one of the best Hyper Parameter Optimization framework out there. Personally, I like Optana for its easy to use (I'll see it in a bit), but there are other great libraries that serves the same purposes very well. Some examples are:
- [Hyperopt](http://hyperopt.github.io/hyperopt/)
- [Ray Tune](https://docs.ray.io/en/latest/tune/index.html)
- [Scikit-Optimize](https://scikit-optimize.github.io/stable/)
- Scikit-learn itself, through [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html).

With some adjustments, the concepts that will be presented are easily extendable for other frameworks as well. So, feel free to use test what I will do here with any other framework you want.